In [0]:
## Generates weighted predictions from trained models

In [0]:
# ## OPTIONAL - download models used to generate the best submission
# !pip -qq install gdown
# !gdown -O models.zip --id 1--Z87pnj5X0RkUiQIYsm0Y5iQCn9xFFU
# !unzip -qq models.zip

In [0]:
import pandas as pd, numpy as np
import sys, os, cv2, random, shutil,time                                                                                                                                                      
import PIL
from fastai.vision import *
from fastai.metrics import error_rate
from fastai.callbacks import SaveModelCallback
from sklearn.model_selection import KFold,StratifiedKFold

import sklearn.metrics as skmetrics

In [0]:
from fastai_extensions import *
from efficientnet_pytorch import EfficientNet
import albumentations as A

In [0]:
RANDOM_STATE = 41
def fix_seed(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
fix_seed(RANDOM_STATE)

In [0]:
DATA_DIR = 'data/'
df_train = pd.read_csv(f'{DATA_DIR}/Train.csv')
df_test = pd.read_csv(f'{DATA_DIR}/Test.csv')
MODEL_DIR = '../../models' ##load models from current directory (relative to data/train)

In [0]:
df_train['path']  = df_train['ID']+'.jpg'
df_train['Label']  = df_train['Label'].astype(np.int8)

In [0]:
def tensor2np(x):
    np_image = x.cpu().permute(1, 2, 0).numpy()
    np_image = (np_image * 255).astype(np.uint8)
    
    return np_image

def alb_tfm2fastai(alb_tfm):
    def _alb_transformer(x):
        # tensor to numpy
        np_image = tensor2np(x)

        # apply albumentations
        transformed = alb_tfm(image=np_image)['image']

        # back to tensor
        tensor_image = pil2tensor(transformed, np.float32)
        tensor_image.div_(255)

        return tensor_image

    transformer = TfmPixel(_alb_transformer)
    
    return transformer()

In [0]:
def get_augs(p=1.0):
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=1.0),
        A.RGBShift(p=0.1, r_shift_limit=(-5, 5), g_shift_limit=(-5, 5), b_shift_limit=(-5, 5)),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=45, p=0.1, border_mode=cv2.BORDER_REFLECT),
        A.OneOf([
            A.MedianBlur(p=1.0, blur_limit=7),
            A.Blur(p=1.0, blur_limit= 7),
            A.GaussianBlur(p=1.0, blur_limit=7),
        ], p=0.05),
        A.OneOf([
            A.ElasticTransform(p=1.0, alpha=1.0, sigma=30, alpha_affine=20),
            A.GridDistortion(p=1.0, num_steps=5, distort_limit=0.1),
            A.OpticalDistortion(p=1.0, distort_limit=0.1, shift_limit=0.5)
        ], p=0.1)
    ],  p=p)


In [0]:
size=200
bs = 16
path = Path(f'{DATA_DIR}/train')

alb_tfms = alb_tfm2fastai(get_augs())
tfms = get_transforms(do_flip=False,
                      flip_vert=False,
                      max_lighting=0.,
                      max_rotate=0.,
                      xtra_tfms=alb_tfms)

##data with albumentations transforms
data = (ImageList.from_df(path=path,df=df_train[['path','Label']])
        .split_none()
        .label_from_df()
        .transform(tfms,size=size)
        .databunch(bs=bs, num_workers=4)
        .normalize(imagenet_stats))

##data for cutmix model with default transforms
data_cmix = (ImageList.from_df(path=path,df=df_train[['path','Label']])
        .split_none()
        .label_from_df()
        .transform(get_transforms(),size=size)
        .databunch(bs=bs, num_workers=4)
        .normalize(imagenet_stats))

In [0]:
def get_preds(model_name):
  learn.load(model_name)
  learn.model.eval()
  preds_test, _ = learn.TTA(ds_type=DatasetType.Test)
  preds_test = preds_test.cpu().detach().numpy()
  preds_test = preds_test[:,1]
  return preds_test

In [0]:
ALL_PREDS = []

In [0]:
pos = np.sum(df_train.Label)
neg = df_train.shape[0] - pos
weights = torch.FloatTensor([1, neg / pos])
weights = weights.to(device)
weights

tensor([1.0000, 2.9132], device='cuda:0')

In [0]:
## B4 model
fix_seed(RANDOM_STATE)
model = EfficientNet.from_pretrained('efficientnet-b4',num_classes=2) 
learn = Learner(data, model,
                loss_func=nn.CrossEntropyLoss(weight=weights),
                metrics=[accuracy],
                callback_fns=[BnFreeze]
               )
learn.split( lambda m: (model._conv_head,) )
learn.data.add_test(ImageList.from_folder(path='data/test/'))
learn.model_dir = MODEL_DIR

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b4-6ed6700e.pth" to /root/.cache/torch/checkpoints/efficientnet-b4-6ed6700e.pth



Loaded pretrained weights for efficientnet-b4


In [0]:
preds = get_preds('effnet4_E0P1')
ALL_PREDS.append(preds)

In [0]:
## B5 model
fix_seed(RANDOM_STATE)
model = EfficientNet.from_pretrained('efficientnet-b5',num_classes=2) 
learn = Learner(data, model,
                loss_func=nn.CrossEntropyLoss(weight=weights),
                metrics=[accuracy],
                callback_fns=[BnFreeze]
               )
learn.split( lambda m: (model._conv_head,) )
learn.data.add_test(ImageList.from_folder(path='data/test/'))
learn.model_dir = MODEL_DIR

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b5-b6417697.pth" to /root/.cache/torch/checkpoints/efficientnet-b5-b6417697.pth



Loaded pretrained weights for efficientnet-b5


In [0]:
preds = get_preds('effnet5_E0P1')
ALL_PREDS.append(preds)

In [0]:
preds = get_preds('effnet5_F1P1')
ALL_PREDS.append(preds)

In [0]:
preds = get_preds('effnet5_F2P1')
ALL_PREDS.append(preds)

In [0]:
preds = get_preds('effnet5_F3P1')
ALL_PREDS.append(preds)

In [0]:
preds = get_preds('effnet5_F4P1')
ALL_PREDS.append(preds)

In [0]:
## B5 cutmix model
fix_seed(RANDOM_STATE)
model = EfficientNet.from_pretrained('efficientnet-b5',num_classes=2) 
learn = Learner(data_cmix, model,
                loss_func=nn.CrossEntropyLoss(weight=weights),
                metrics=[accuracy],
                callback_fns=[BnFreeze]
               ).cutmix()
learn.split( lambda m: (model._conv_head,) )
learn.data.add_test(ImageList.from_folder(path='data/test/'))
learn.model_dir = MODEL_DIR

Loaded pretrained weights for efficientnet-b5


In [0]:
preds = get_preds('effnet5_cmix_F0P1')
ALL_PREDS.append(preds)

### Submission

In [0]:
WEIGHTS = [0.149412, 0.150477, 0.135532, 0.140804, 0.142985, 0.134682, 0.146108] 
WEIGHTED_PREDS = []
for i in range(len(ALL_PREDS)):
  WEIGHTED_PREDS.append(ALL_PREDS[i]*WEIGHTS[i])

In [0]:
preds = np.asarray(WEIGHTED_PREDS)
preds = preds.sum(axis=0)
preds.shape

(2613,)

In [0]:
df_submission = pd.DataFrame()
df_submission['ID'] = [s.split('.')[0] for s in os.listdir(f'{DATA_DIR}/test')]
df_submission['Label'] = preds
df_submission.head()

,ID,Label
0,MXG6CR6,0.180806
1,KST49H6,0.142675
2,AHIISOM,0.241601
3,9S0MQOK,0.205762
4,X0B7JYR,0.281761


In [0]:
df_submission.Label.min(),df_submission.Label.max()

(0.00029082587570883334, 0.9892255663871765)

In [0]:
df_submission.to_csv('ensemble_wtd.csv',index=False)